In [5]:
import praw
import pandas as pd
from praw.models import MoreComments
import datetime as dt
from datetime import date

In [53]:
payload=pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')
first_table = payload[0]
second_table = payload[1]
thr = payload[2]
thr.head()
symbols = thr['Symbol'].values.tolist()
symbols = [i.lower() for i in symbols]
# # symbols = set(symbols)
names = thr['Name'].values.tolist()
names = [i.lower() for i in names]

helper = {}
for i in range(len(symbols)):
    helper[symbols[i]] = names[i]

symbols.remove("so")
symbols.remove("low")
symbols.remove("f")



In [64]:
# input : Set of String, Integer, String
# Set of String = Keywords
# Integer = Postcount to evaluate
# String = Subreddit Name
# Boolean - check if data should be written
def redditFilterHot(tsla, post_count, subreddit, txt):
    #using API key create instance of Reddit for PRAW
    reddit = praw.Reddit(client_id="H2yrxHGdrXYUxexBul36Zw", client_secret = 'mouzzGMwsNMquUX4EA-GF_9BjAIyvA', user_agent = "wsbWebScraping")
    #selecting subreddit
    sr = reddit.subreddit(subreddit)
    
    
    # Create Dataframe
    posts = []
    for post in sr.hot(limit=post_count):
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
    posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
    
    
    # Open File to Add Data to text file
    if (txt):
        today = date.today()
        use = today.strftime("%d-%m-%y")
        file = open(f"{use}.txt", mode="w+")
        file.write("Author,Body")
    
    # Adding DataFrame
    dictionary = {"Author":[], "Body":[], "Upvotes":[], "Tag":[]}
    
    
    #parse through every post in dataframe
    # TODO : optimize -- likely more efficient way to access posts directly through dataframe
    for num in range(len(posts["url"])):
        # parse through all comments --> including subcomments
        # utilize try block --> avoid invalid URLs
        try: 
    #opens post
            submission = reddit.submission(url = posts["url"][num])
    #skip reddit buttons that are not posts
            for c in submission.comments.list():
                if isinstance(c, MoreComments):
                    continue
    #comparing sets to see if common keywords
                if not tsla.isdisjoint(set(c.body.lower().split(" "))):
    #cleaning up print
    #TODO - Make it more clean
    #printing disabled for now
                    #print(f'''
#______________________________________________________________
#{c.author}:
#--------------------------------------------------------------
#{c.body}
#______________________________________________________________
                    #''')
        #Write to TXT File
                    #if txt:
                        #file.write(f"{c.author},{c.body}\n")
                    
                    # DataFrame
                    dictionary["Author"].append(c.author)
                    dictionary["Body"].append(c.body)
                    dictionary["Upvotes"].append(c.score)
                    dictionary["Tag"].append(tsla.intersection(set(c.body.lower().split(" "))))
        except:
            pass
    df = pd.DataFrame(dictionary)
    if txt:
        file.write(df.to_csv())
    return df
    
    

In [69]:
tsla = {"tesla", "tsla", "elon"}
tsla = set(symbols).union(set(names))
pc = 50
sr = "wallstreetbets"
redditFilterHot(tsla, pc, sr, True)

,Author,Body,Upvotes,Tag
0,FU_money_pharm17,Currently sitting in my Tesla on the side of a...,30,{tesla}
1,SendMeHawaiiPics,Listen to the fucking market. Walmart and Targ...,25,"{target, walmart}"
2,iop9,I have severely bad news for TSLA bulls... Cat...,20,{tsla}
3,RBlankenship,“Everyone uses Microsoft teams now”![img](emot...,13,{microsoft}
4,WallStreetlegend_1,I went to Walmart for chicken breast and black...,11,{walmart}
...,...,...,...,...
73,recurse_x,Yeah but its disguised as a TikTok soft sell P...,3,"{costco, tesla}"
74,Onebadmuthajama,It's all fine until the music stops playing. ...,0,{cost}
75,Das_Siegfried,Where's that guy with the DD about monkeypox s...,1,{dd}
76,PUMK1ng,Have a sensible stoploss and trail it when you...,1,{target}
